In [1]:
import pandas as pd
from datetime import date

## Time Slots

In [2]:
# Time Slots
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat']
n_days = len(days)
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']
n_times = len(times)
lunch_time_index = times.index('13:00')
pre_lunch_times = list(range(lunch_time_index + 1))
post_lunch_times = list(range((lunch_time_index + 1), len(times)))

def begin_end_pairs(session_len:int) -> list:
    pairs = []
    for s in range(0, len(times) - session_len):
        e = s + session_len
        if (((s in pre_lunch_times) and (e in pre_lunch_times)) or
            ((s in post_lunch_times) and (e in post_lunch_times))):
            pairs.append((s,e))
    return pairs
# No lectures start/end in these time slots (as (day, time) tuples) --- labs, tutorials are OK
mon_2pm = (days.index('Mon'), times.index('14:00'),)
wed_2pm = (days.index('Wed'), times.index('14:00'),)
no_lecture_slots = ([(mon_2pm[0], t,) for t in range(mon_2pm[1], (mon_2pm[1] + 6))] +
                    [(wed_2pm[0], t,) for t in range(wed_2pm[1], (wed_2pm[1] + 6))])

def overlap(b:int, e:int, t:int) -> bool:
    # Returns a bool indicating whether the intervel (b,e) contains time t
    # b,e,t are indexes into the 'times' array
    assert (0 <= b < n_times) and(0 <= e < n_times) and (0 <= t < n_times)
    return (b <= t <= e)



## Classrooms

In [3]:
# classroom_cols = {'RoomNum': str, 'Capacity': int, 'Category': str,
#                   'HasProjector': bool, 'HasWacom': bool}
classroom_cols = {'RoomNumber': str, 'Capacity': int}
classrooms = pd.read_csv('classroom_master.csv', dtype=classroom_cols)
n_rooms = classrooms.shape[0]

#get room capacity given room id 
def get_room_capacity(room_id: int) -> int:
    return int(classrooms.loc[room_id, 'Capacity'])

def get_room_category(room_id: int) -> str:
    a=classrooms.loc[room_id, 'Lab']
    b=classrooms.loc[room_id, 'DeptLab']
    if a:
        if b:
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_room_num(room_id: int) -> str:
    return classrooms.loc[room_id, 'RoomNumber']

def check_cse_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'CSE Lab':
        return True
    return False

def check_ece_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'ECE Lab':
        return True
    return False


## Courses & Enrollment

In [4]:
courses = pd.read_csv('course_master.csv')
n_courses = courses.shape[0]
enrollment = pd.read_csv('enrollment_master.csv')
n_enrollments = enrollment.shape[0]
# number of unique emails in enrollment_master_clean.csv
n_students = enrollment['EmailID'].nunique()
students = enrollment['EmailID'].unique()
print(n_students)

# # Write students to a csv file 
# students_df = pd.DataFrame(students, columns=['EmailID'])
# students_df.to_csv('students.csv', index=False)

course_id = {}
for i in range(n_courses):
    course_id[courses['Code'][i]] = i

course_id_code={}
for i in range(n_courses):
    course_id_code[i] = courses['Code'][i]


def get_course_id(course_code: str) -> int:
    return course_id[course_code.strip()]

# make function to get ccourse strength given course id
def get_course_strength(course_id: int) -> int:
    return int(enrollment[enrollment['CourseCode'] == courses['Code'][course_id]].shape[0])
def get_courses_student(student_id: int) -> list:
    l =  enrollment[enrollment['EmailID'] == students[student_id]]['CourseCode'].values
    return [get_course_id(x) for x in l]

def get_course_category(course_id: int) -> str:
    
    a = courses.loc[course_id,'Lab']
    b = courses.loc[course_id,'Dept']
    if a:
        if b == 'CSE':
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_not_rooms_course(course_id: int) -> list:
    l=[]
    print("Course:",course_id)
    for i in range(n_rooms):
        if get_room_category(i) != get_course_category(course_id):
            l.append(i)
    print("Rooms:",l)
    return l


# for x in students:
#     print(x)

# for c in range(n_courses):
#     print("Course:",c)
#     print("Course Type:",get_course_category(c))
#     print(get_not_rooms_course(c))


def get_course_duration(course_id: int) -> int:
    return courses.loc[course_id, 'Exam Duration']


def get_preferred_slots(course_id: int) -> list:
    x = courses.loc[course_id, 'Preferred Timings'].split(',')
    # print(x)
    triplets = []
    for i in range(0, len(x), 3):
        begin = x[i].strip(" [(\"'")
        end = x[i+1].strip(" )'\"]")
        day = x[i+2].strip(" )'\"]")
        triplets.append((begin, end, day))

    # print(triplets)

    #Converting them into indxes
    for i in range(len(triplets)):
        triplets[i] = (times.index(triplets[i][0]), times.index(triplets[i][1]), days.index(triplets[i][2]))
    # print(triplets)
    return triplets

# get_preferred_slots(0)


973


In [5]:

from collections import defaultdict

course_student = defaultdict(list)
intersection_pairs = set()
intersection_triplets = set()
for i in range(n_students):
    courses_taken = get_courses_student(i)
    for x in courses_taken:
        course_student[x].append(i)
    # add all possible course pairs to intersection_pairs
    for j in range(len(courses_taken)):
        for k in range(j+1, len(courses_taken)):
            intersection_pairs.add((courses_taken[j], courses_taken[k]))
            # add all possible course triplets to intersection_triplets
            for l in range(k+1, len(courses_taken)):
                intersection_triplets.add((courses_taken[j], courses_taken[k], courses_taken[l]))
    
#get unique pairs
intersection_pairs = list(intersection_pairs)
n_intersection_pairs = len(intersection_pairs)
print(intersection_pairs)
print(course_student)
print(intersection_triplets)

[(15, 30), (7, 26), (40, 41), (48, 54), (11, 23), (41, 42), (10, 36), (10, 45), (2, 41), (10, 54), (3, 24), (3, 33), (3, 42), (14, 51), (36, 48), (47, 48), (7, 21), (17, 53), (9, 49), (29, 36), (48, 49), (6, 43), (29, 54), (10, 31), (10, 49), (22, 23), (22, 32), (14, 37), (55, 56), (5, 55), (29, 31), (2, 22), (25, 33), (22, 27), (3, 23), (13, 55), (16, 51), (28, 34), (17, 34), (36, 47), (9, 30), (9, 48), (21, 22), (6, 33), (10, 30), (2, 26), (24, 32), (35, 41), (14, 18), (16, 37), (17, 38), (28, 38), (21, 26), (39, 43), (2, 3), (24, 27), (3, 4), (51, 53), (1, 34), (16, 50), (5, 22), (9, 20), (9, 29), (9, 47), (6, 32), (2, 7), (20, 43), (12, 48), (4, 44), (16, 18), (1, 38), (35, 40), (5, 8), (17, 19), (38, 51), (17, 28), (5, 26), (5, 44), (27, 41), (50, 52), (8, 55), (20, 29), (39, 42), (23, 25), (23, 43), (4, 39), (1, 15), (24, 26), (32, 39), (5, 12), (15, 53), (9, 28), (8, 32), (42, 43), (11, 46), (12, 20), (12, 29), (4, 25), (12, 47), (4, 43), (1, 19), (34, 53), (1, 28), (5, 7), (46,

In [6]:
def get_intersection_of_courses(c1, c2):
    return (set(course_student[c1]) & set(course_student[c2]))

## Faculty

In [7]:
faculty_cols = {'Name': str, 'Course': str, 'Start': str, 'End': str,
                'PreferredSlot': str, 'Flexibility': int}
faculty = pd.read_csv('faculty_master.csv', dtype=faculty_cols)